In [ ]:
import numpy as np
import gym
import random

# Load the Taxi-v3 environment
env = gym.make("Taxi-v3")

# Q-learning parameters
alpha = 0.1       # Learning rate
gamma = 0.6       # Discount factor
epsilon = 0.1     # Exploration rate

# Initialize Q-table with dimensions matching observation and action space
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Training the agent
num_episodes = 10000
for i in range(num_episodes):
    # Reset environment and retrieve initial state based on Gym version
    state = env.reset() if isinstance(env.reset(), int) else env.reset()[0]
    
    epochs, penalties, reward = 0, 0, 0
    done = False

    while not done:
        if random.uniform(0, 1) < epsilon:
            # Explore: select a random action
            action = env.action_space.sample()
        else:
            # Exploit: select the action with max value (greedy)
            action = np.argmax(q_table[state])

        # Take action in the environment and unpack values
        step_result = env.step(action)
        next_state, reward, done, info = step_result[:4]

        # Update Q-value
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        # Count penalties
        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1

print("Training finished.\n")

# Evaluating the agent
total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    # Reset environment and retrieve initial state
    state = env.reset() if isinstance(env.reset(), int) else env.reset()[0]
    
    epochs, penalties, reward = 0, 0, 0
    done = False

    while not done:
        action = np.argmax(q_table[state])  # Choose action based on the Q-table
        step_result = env.step(action)
        state, reward, done, info = step_result[:4]  # Unpack only the first four values

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")